### CAP 6640 
### Project 2 - Fake News Detection
### Mar 28, 2024

### Group 4
### Andres Graterol
###                   UCF ID: 4031393
### Zachary Lyons
###                   UCF ID: 4226832
### Christopher Hinkle
###                   UCF ID: 4038573
### Nicolas Leocadio
###                   UCF ID: 3791733

#### Imports

In [65]:
import re 
import nltk
import gensim
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
# Importing Libraries w.r.t Word Embedding layer & Lstm NN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# Download necessary resources from nltk
try:
    nltk.data.find('stopwords')
except LookupError:
    nltk.download('stopwords')
    
try:
    nltk.data.find('punkt')
except LookupError:
    nltk.download('punkt')
    
try:
    nltk.data.find('wordnet')
except LookupError:
    nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nick_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Step 1 - Data Collection

In [66]:
# Read in the training dataset as a dataframe
df = pd.read_csv('train.csv')

# Drop NaN rows from the dataframe to avoid errors
df = df.dropna(how='all', axis=0)

# Merge the author and title into a single column
df['article'] = df['author'] + ' ' + df['title']
# print(df['article'])

# print(df)

#### Step 2 - Data Preprocessing

In [67]:
df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [68]:
# Combine title and author into a single column, diverting from tutorial
# df = df.drop('title', axis=1)
# df = df.drop('author', axis=1)
df.head()

,id,title,author,text,label,article
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [69]:
# Split data in into data (X) and labels (Y)

# Seclude the labels
# 1 - unreliable
# 0 - reliable
labels = df['label']
#print(labels)

# Get the data excluding the labels
data = df.drop('label', axis=1)
# print(data)

#####   COPY PASTA DOWN BELOW
##### COOK THE SPAGHETTI HARDER

In [70]:
# ps = PorterStemmer()
# corpus = []
# for i in range(0, len(df)):
#     review = re.sub('[^a-zA-Z0-9]',' ', str(df['title'][i]))
#     review = review.lower()
#     review = review.split()

#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

In [71]:
# Only having Lower & Upper Case Alphabets & Numbers, Removing Stop Words, Applying Lemmatization

lemmatizer = WordNetLemmatizer()

cleaned_corpus = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z0-9]',' ',str(df['article'][i]))
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    cleaned_corpus.append(review)

In [72]:
# similarly Dropping Those rows in Y w.r.t Corpus which were empty Strings
labels = df[list(map(lambda x: len(x)>0,cleaned_corpus))]
labels = labels['label']

In [73]:
# Removing empty Sentences from Corpus w.r.t Corpus
cleaned_corpus = [i for i in cleaned_corpus if i]
len(cleaned_corpus)

20771

In [ ]:
voc_size=5000

onehot_repr=[one_hot(words,voc_size)for words in cleaned_corpus]